In [1]:
#!/usr/bin/env python
# coding: utf-8

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from langdetect import detect_langs
import pandas as pd
import time
import random
import requests

In [2]:
import import_ipynb
import config

In [ ]:
import import_ipynb
import Model

importing Jupyter notebook from Model.ipynb


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
number_of_chats=0
pipelines_dict = {}

In [ ]:

# We open Chrome browser in incognito mode and maximized
def open_browser():
    service = ChromeService(executable_path=ChromeDriverManager().install())

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--start-maximized")

    driver = webdriver.Chrome(service=service, chrome_options=chrome_options)
    return driver


# Link to the desired page to be scraped
def navigate(driver):
    driver.get('https://www.linkedin.com/login/es?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin');
    time.sleep(10)

    # Save all the content in the "soup" variable
    content = driver.page_source
    soup = BeautifulSoup(content, "html.parser")


#get and insert login credentials
def login(driver):
  
    # Login
    # Getting the username element 
    username_box = driver.find_element_by_id("username")  
    username_box.clear()
    time.sleep(1)

    # Sending the keys for username     
    username_box.send_keys(config.username) 
    time.sleep(1)


    # Getting the password element                            
    password_box = driver.find_element_by_id("password") 
    password_box.clear()
    time.sleep(1)

    # Sending the keys for password   
    password_box.send_keys(config.password) 
    time.sleep(1)

    # Getting the tag for submit button           
    driver.find_element(By.XPATH, "//button[@aria-label='Iniciar sesión']").click()
    time.sleep(10)


#Search IT recruiters
def search(driver):
    search_box=driver.find_element(By.XPATH, "//input[@placeholder='Buscar']")
    try:
        search_box.clear()
    except:
        print("No content to clear")

    search_box.send_keys("recruiter meta")
    time.sleep(5)
    
    #Click see all contacts
    driver.find_element(By.XPATH, "//span[@class='search-global-typeahead__hit-text t-16 t-black']").click()
    time.sleep(5)
    
    #set filter to person
    try:
        driver.find_element(By.XPATH, "//button[@aria-label='Personas']").click()
        time.sleep(1)
    except:
        driver.find_element(By.XPATH, "//*[@id='search-reusables__filters-bar']/ul/li[1]/button").click()
        time.sleep(1)


# Save all the content in the "soup" variable
def get_page_content(driver):
    content = driver.page_source
    soup = BeautifulSoup(content, "html.parser")


def send_invitations(num, driver):
    
    i=0
    contacted=0
    while contacted<int(num):
        content = driver.page_source
        soup = BeautifulSoup(content)
        new = soup.findAll('div', {'class':'entity-result'})

        # Iterates over all the persons in this page (10)
        for item in new:
            print("######################" + "Number: "+ str(i) + " Person"  +   "###########################")
            
            #location=driver.find_element(By.XPATH,"entity-result__secondary-subtitle t-14 t-normal")
            soup=get_content(driver)
            print("RECRUITER FROM")
            print(soup.find('div',{'class':'entity-result__secondary-subtitle t-14 t-normal'}).get_text())

            #Press connect button, random time (to imitate human behaviour)
            time.sleep(random.randint(2, 7))
            connect_button=None
            follow_button=None
            message_button=None
            try:
                #connect_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-button artdeco-button--2 artdeco-button--secondary ember-view') and contains(., 'Conectar')]")
                connect_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-button artdeco-button--2 artdeco-button--secondary ember-view') and contains(., 'Conectar')]")
                #connect_button.click()
                connect=1
                contacted=contacted+1
                print("Connect button detected")
            except:
                try:
                    follow_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-button artdeco-button--2 artdeco-button--secondary ember-view') and contains(., 'Seguir')]")
                    follow_button.click()
                    print("Follow button detected")
                except:
                    try:
                        message_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-button artdeco-button--2 artdeco-button--secondary ember-view') and contains(., 'Mensaje')]")
                        #message_button.click()
                        print("Message button detected")
                    except:
                        if connect_button==None and follow_button==None and message_button==None:
                            print("[+] No button detected\n")

            #print(conectar_button)
            time.sleep(random.randint(2, 7))

            #Press send invitation
            try:
                enviar_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml1') and contains(., 'Enviar')]")
                #enviar_button.click()
                print("Invitation sent")
                
                #To be sure that the button pressed before this was "connect"
                if connect==1:
                    contacted=contacted+1
            except:
                print("[+] No 'send' button \n")

                
            time.sleep(2)
            #print(enviar_button)

            if contacted<int(num):
                # Keep going
                i=i+1
            else:
                #Number of recruiters contacted reached
                break
                
        #Scroll down to click next
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Save all the content in the "soup" variable
        content = driver.page_source
        soup = BeautifulSoup(content)

        
        try:
            next_page_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-pagination__button artdeco-pagination__button--next artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view') and contains(., 'Siguiente')]")        
            next_page_button.click()
        except:
            print("error")


def goto_messages(driver):
    try:
        messages=driver.find_element(By.LINK_TEXT, "Mensajes")
        messages.click()
    except:
        messages=driver.find_element(By.XPATH, "//*[@id='global-nav']/div/nav/ul/li[4]/a/span")
        messages.click()


def get_content(driver):
    content = driver.page_source
    soup = BeautifulSoup(content, "html.parser")
    return soup


def add_to_exclusion_list(name):
    f = open("exclusion_list.txt", "a")
    f.write("\n"+name)
    f.close()
    

def get_from_exclusion_list():
    exclusion_list=[]
    file1 = open('exclusion_list.txt', 'r')
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        exclusion_list.append(line.strip())
    
    return exclusion_list


def in_exclusion_list(name):
    exclusion_list=[]
    file1 = open('exclusion_list.txt', 'r')
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    for line in Lines:
        if line.strip()==name:
            return True
    
    return False



def del_from_exclusion_list(name):
    lines=[]
    lines=get_from_exclusion_list() #get all excluded members
    lines.remove(name)
        
    
    file = open("exclusion_list.txt", "w") #rewrite everything
    for line in lines:
        file.write(line + "\n")
    
    file.close()



def send_telegram_alert(status, url, name):
    bot_message=None
    if status=="ON":
        bot_message="🟢 "+name+" has activated the BOT again"
    elif status=="OFF":
        bot_message="🔴 "+name+ " wants to talk directly with you! \nReply here: ⬇️ ⬇️ ⬇️\n" +url

    bot_token = 'xxxxxxxxxxxxxxxxxxxxxxxxx'
    bot_chatID = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    print(response.json())






def check_bot_status(status, url, name, driver):
    
    #if any of the messages contains "start bot" delete name from the exclusion list
    if "on" in status.lower():
        # Enviar mensaje a telegram
        
        if in_exclusion_list(name)==True: #user in exclusion list so we can delete it
            del_from_exclusion_list(name)
            get_content(driver)
            send_telegram_alert("ON", url, name)
            print("__________________________ "+name+ " DELETED FROM EXCLUSION LIST __________________________")
        else:
            print("__________________________ "+name+ " ISNT IN EXCLUSION LIST __________________________")
        
    #if last message is stop bot, add member to exclusion list
    elif "off" in status.lower():
        # Add name to the exclusion list
        
        if in_exclusion_list(name)==False: #user not in exclusion list so we can add it
            add_to_exclusion_list(name)
            get_from_exclusion_list()
            get_content(driver)
            send_telegram_alert("OFF", url, name)  
            print("__________________________ "+name+ " ADDED TO EXCLUSION LIST __________________________")
        else:
            print("__________________________ "+name+ " ALREADY IN EXCLUSION LIST __________________________")



def reply_private_message(response, driver):

    # Getting the reply element                            
    reply_box = driver.find_element(By.XPATH, "//div[@aria-label='Escribe un mensaje…']") 
    
    print(reply_box)

    #reply_box=driver.find_element(By.LINK_TEXT, "Escribe un mensaje")
    #password_box.clear()
    time.sleep(1)
    

    # Sending the keys for password   
    reply_box.send_keys(response) 
    reply_box.send_keys() 
    
    time.sleep(1)
    
    send_button=driver.find_element(By.XPATH, "//button[@type='submit']") 
    send_button.click()
    
    print("MESSAGE SENT: ")
    print(response)




def language_detection(text):

    language=detect_langs(text)
    print(language)
    first_language=language[0]
    
    language, score=str(first_language).split(":")
    return language


def add_connect_message(driver):
    add_note_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-button artdeco-button--muted artdeco-button--2 artdeco-button--secondary ember-view mr1') and contains(., 'Añadir una nota')]")
    #add_note_button.click()
                
    time.sleep(1)
    
def send_connect_message(driver):    
    # Getting the password element                            
    text_box = driver.find_element(By.ID, "custom-message") 
    print(text_box)
    text_box.send_keys(config.username)
    
    enviar_button=driver.find_element(By.XPATH,"//button[contains(@class, 'artdeco-button artdeco-button--2 artdeco-button--primary ember-view ml1') and contains(., 'Enviar')]")
    enviar_button.click()
    print("Invitation sent")
    

In [ ]:
def check_number_of_chats():
    goto_messages(driver)
    # Save all the content in the "soup" variable
    goto_messages(driver)
    soup=get_content(driver)
    
    # Save all the "li" so we can iterate over all the different chats
    new = soup.findAll('li', {'class':'scaffold-layout__list-item msg-conversation-listitem msg-conversations-container__convo-item msg-conversations-container__pillar ember-view'})
    return len(new)

def save_pipeline_list(pipeline_list):
    saved_pipeline_list=[]
    saved_pipeline_list=pipeline_list
    return saved_pipeline_list

def initiate_all_pipelines(pipelines_dict):
    # iterate over the conversations to
    goto_messages(driver)
    # Save all the content in the "soup" variable
    goto_messages(driver)
    soup=get_content(driver)
    
    # empty dictionary

    # Save all the "li" so we can iterate over all the different chats
    new = soup.findAll('li', {'class':'scaffold-layout__list-item msg-conversation-listitem msg-conversations-container__convo-item msg-conversations-container__pillar ember-view'})
    for item in new:
        name=item.h3.get_text()
        name=name.strip()
        name=name.replace(" ", "_")
        
        
        if name not in pipelines_dict:
            pipeline_name = Model.initiate_conversational_pipeline(name)
            pipelines_dict[name] = pipeline_name
    
    return pipelines_dict



def get_messages2(driver, pipelines_dict):
    goto_messages(driver)
    # Save all the content in the "soup" variable
    goto_messages(driver)
    soup=get_content(driver)
    
    exclusion_list = []
    previus_comment=""
    
    # Save all the "li" so we can iterate over all the different chats
    new = soup.findAll('li', {'class':'scaffold-layout__list-item msg-conversation-listitem msg-conversations-container__convo-item msg-conversations-container__pillar ember-view'})
    for item in new: 
        
        initiate_pipeline=0
        
        # Every item is a different chat with a different person
        name=item.h3.get_text()
        name=name.strip()
        print(" ########################### CONVERSATION WITH "+name+" #########################")
        if name not in pipelines_dict: # There isnt conversational pipeline for this person
            pipeline_list=initiate_all_pipelines(pipelines_dict)
            
        # Dynamically created Xpath    
        href_text="//h3[normalize-space()="+str("\'")+name+str("\'")+str("]")
           
        #Based on the href reference go to the private conversation
        try:
            show_message_button=driver.find_element(By.XPATH, href_text)        
            show_message_button.click()
            time.sleep(1)
        except:
            print("Error: No more elements")

        time.sleep(2)
        soup=get_content(driver)
        
        name_div=None
        send_message=0    

        #if "quick replies" div exists, means that i have to answer
        name_div=soup.find('div', {'class':'conversations-quick-replies justify-center display-flex align-items-center'})

        if name_div is None:
            print("__________________________ SKIP THIS CONVER, LAST MESSAGE IS ALREADY MINE __________________________")
        else:
            print("__________________________ I HAVE TO RESPOND __________________________")
            send_message=1
                
        slide_message=""
        whole_message=""
        
        # If im in this loop, at least the last message must be fron the other person
        if send_message==1: #Last message isnt mine, i have to respond
        # First, analize the "slide in" div   
            
            # Get the url to send to the telegram, if the user types "stop bot" i will receive a message with the url 
            # si i can answer it via linkedin app in the mobile
            url=driver.current_url
        
            result=0
            
            try:
                print("__________________________ SLIDE IN (CSS protected div) __________________________")
                result=None
                result=soup.find('li',{'class':'msg-s-message-list__event clearfix msg-s-message-list__event--slide-in'})

                slide_message=result.find('p',{'class':'msg-s-event-listitem__body t-14 t-black--light t-normal'}).get_text()
                print(slide_message)
                
                result=check_status(slide_message, url, name, driver)
                
                if result==False:  # If last message isnt stop/start, save it
                    print("Normal message")
                    slide_message=slide_message+str(".|.")
                    
                elif result==2:    # last message is stop
                    print("STOP MESSAGE")
                else:              # last message is start
                    print("START MESSAGE")
                    
            except:
                print("cannot get slide in message (probably is a gif, emoji...)")
            

            try:
                message_sender=result.find("a",{'class':'msg-s-message-group__profile-link ember-view'}).get_text().strip()
                print("Message sent by: "+message_sender)
               
                # Save it
                # Is the last message, could be the first one or the others were sent at a different time
                #whole_message=str(last_message)+str(".|.")

                
            except:
                # Is the last message but the sender sent more
                print("Message is a continuation")
                #whole_message=str(last_message)+str(".|.")

            
            if result==2 or result==1: #if the last message is start/stop, end we dont need anything
                pass
            else:
                
                ### Now its time to analize all the conversation, from bottom to top, if the messages are sent at a different
                # time, there will be different divs so we have to analyze them all until we get our name, that means that the 
                # last message is already analyzed

                # this loop iterates over all messages in the chat with a concrete person
                # Last message already analized


                li_divs=soup.findAll('li', {'class':'msg-s-message-list__event clearfix'})
                whole_message=""
                for div in reversed(li_divs):
                    time.sleep(1)
                    soup=get_content(driver)

                    message_sender=None

                    print("__________________________ NORMAL __________________________")
                    try:
                        message_sender=div.find("a",{'class':'msg-s-message-group__profile-link ember-view'}).get_text().strip()

                        if message_sender != "Mikel Uranga":
                            print("Message sent by: "+message_sender)

                            try:
                                normal_message=div.find('p',{'class':'msg-s-event-listitem__body t-14 t-black--light t-normal'}).get_text()
                                print(normal_message)
                                
                                inner_result=check_status(normal_message, url, name, driver)
                                if inner_result==False:
                                    whole_message=str(normal_message)+str(".|.")+whole_message
                                elif inner_result==2: #stop!
                                    print("STOP")
                                    break
                                else:
                                    print("START")
                                    break
                                
                            except:
                                print("could get message")
                        else:
                            # This means that the sender is me again, so i must save my comment to improve the translation and
                            # exit the iteration and stop analyzing
                            
                            previus_comment=div.find('p',{'class':'msg-s-event-listitem__body t-14 t-black--light t-normal'}).get_text() 
                            break
                            
                    except:
                        print("no message sender, Message is a continuation")
                        try:
                            normal_message=div.find('p',{'class':'msg-s-event-listitem__body t-14 t-black--light t-normal'}).get_text()
                            inner_result=check_status(normal_message, url, name, driver)
                            if inner_result==False:
                                whole_message=str(normal_message)+str(".|")+whole_message
                            elif inner_result==2: #stop!
                                print("STOP")
                                break
                            else:
                                print("START")
                                break
                        except:
                            print("could get message")

                    print(whole_message+slide_message)

                

                if whole_message == "" and result==1: #Status is start and there is no more messages
                    pass 
                else:
                    
                    previus_comment=whole_message

                # get the users in the exclusion list (those who asked for a manual conversation)
                    exclusion_list=get_from_exclusion_list()
                    
                    last_message=whole_message+slide_message

                    if name not in exclusion_list:
                        print("__________________________ ULTIMO MENSAJE RECIBIDO  __________________________")
                        print(last_message)

                        # Get laguage
                        language=language_detection(last_message)

                        #if the text is in spanish, translate to english and send to the model (model works in english)
                        #then translate the reply again to spanish
                        
                        # Get value of pipeline for the given name
                        pipeline_key=name.replace(" ", "_")
                        pipeline=pipeline_list.get(pipeline_key)

                        if language=="en":
                            # Delete all the "|" and send directly to model

                            #translating
                            print("Sending message to model")
                            #last_message=last_message.replace("|", "")

                            response=None
                            response = Model.generate_response(last_message, pipeline)
                            print("Response is"+response)

                            reply_private_message(response, driver)

                        else:


                            # ignore start bot and stop bot
                            print("translating from spanish")
                            eng_response=None
                            eng_response=Model.translate_es_en(last_message)

                            print(eng_response)

                            english_response=None
                            english_response = ''.join(eng_response)
                            
                            # This is the question
                            print(english_response)

                            response=None
                            print("Sending message to model")
                            
                            # This is the response
                            response = Model.generate_response(english_response, pipeline)
                            
                            print(response)
                            response=response.replace("|", "")

                            esp_response=None
                            print("Translating to spanish again")
                            #Send question an response, so the context is better and it will translate better too

                            # /#/ used to divide the question and the response, other type of delimitation characters are deleted
                            # by the translation model
                            esp_response=Model.translate_en_es(english_response+"/#/"+response)
                            esp_response_splitted=esp_response[0].split("/#/")
    
                            #translate result to spanish
                            reply_private_message(esp_response_splitted[1], driver)
            
        

            
def check_status(last_message, url, name, driver):
    # Return True if message is either start bot/stop bot and false if is any other thing
    if "start" in last_message.lower() and "bot" in last_message.lower():
        check_bot_status("ON", url, name, driver)
        print("MESSAGE IS START")
        return 1
        
    elif "stop" in last_message.lower() and "bot" in last_message.lower():
        check_bot_status("OFF", url, name, driver)
        print("MESSAGE IS STOP")
        return 2
    
    else: # No stop/start, then get message to translate
        print("NO START/STOP")
        return False
        